# Teaching AI Agents to Use Tools - Adding Actions to your code assistant (Part 2)

## Introduction
 So far, our **AI Code Assistant** has learned how to think - it can read code and make intelligent suggestions.

 But it still can't do anything.  
 It just talks about code instead of working with code.

 In this part, we will change that by introducing **tools** - external actions your agent can perform beyond text generation.

 By the end of this tutorial, your assistant will:
 * Choose between **analyzing** code or **reading** from a file,
 * Dynamically decide what to do,
 * And execute the right tool to complete the job

 ## Concept: Tools are Actions
 In the real worlds, a human code reviewer might:
 * Open files
 * Run tests
 * Edit code

 LLMs cannot directly do that.
 With tool calling, we can connect them to real Python functions that perform these tasks safely.

## The Tool Registry
We will give out agent multiple tools, like:


 | Tool Name      | Description                      | Function                  |
 | -------------- | -------------------------------- | ------------------------- |
| `read_file`    | Read code from a file            | `def read_file(path)`      |
| `analyze_code` | Analyze a string of code         | LLM reasoning via API call |
| `write_tests`  | Generate and save test functions | `def write_tests(code)`    |

The agent decides; "Do I need to read a file, write a test or analyze the given snippet?"

Let's go ahead and build some tools, and give the to the code assistant that decides which tool to use based on user instruction and run the tool.


In [3]:
import openai
import os
from typing import Callable, Dict

### Step 1 Define some tools

In [2]:
def read_file(filepath: str) -> str:
    """Read contents of a Python file"""
    if not os.path.exists(filepath):
        return f"File not found: {filepath}"
    
    with open(filepath, "r") as f:
        return f.read()

def write_tests(test_code: str) -> str:
    test_file_path = "./tests/generated_test.py"

    with open(test_file_path,"w") as f:
        f.write(test_code)
    
    return f"Wrote test file: {test_file_path}"

def analyze_code(code: str) -> str:
    """Ask an LLM to analyze the provided code."""
    prompt = f"""
    You are a helpful code review assistant.
    Analyze the following Python code and suggest one improvement.

    Code:
    {code}
    """

    ressponse = openai.responses.create(model="gpt-4.1-mini",input=[{"role":"user","content":prompt}])

    return ressponse.output_text

### Step 2: Build a simple tool registry

In [4]:
class ToolRegistry:
    """Holds available tools and dispatches them by name."""
    def __init__(self):
        self.tools: Dict[str,Callable] = {}
    
    def register(self, name:str, func: Callable):
        self.tools[name] = func

    def call(self, name:str, *args, **kwargs):
        if name not in self.tools:
            return f"Unknown tool: {name}"
        return self.tools[name](*args, **kwargs)

### Step 3: Agent with tool use

**NOTE:** Not all LLM models have function calling and structured output capabilities. For models that dont support function calling this capability to be achieved with well structured prompts and instructions.

We will start by building a simple tool agent and then extend that to show how to use LLM model function calling and structured output functionality for LLM models that support it.

In [12]:
class CodeAssistantWithTools:
    def __init__(self, tool_registry: ToolRegistry, model= "gpt-4o-mini"):
        self.tools = tool_registry
        self.model = model
    
    def decide_tool(self, user_input: str):
        """LLM decides which tool to use"""

        prompt = f"""
        You are a code assistant with access to the tools below.

        Available tools:
        - read_file(filepath)
        - analyze_code(code)
        - write_tests(test_code)

        Decide which tool is most appropriate based on the user input below.
        Reply ONLY with the tool name and argument if needed.

        Examples:
        read_file("main.py")
        analyze_code("def foo():pass")
        write_tests("def foo():pass")

        user input: {user_input}
        """

        response = openai.responses.create(model=self.model, input=[{"role":"user","content":prompt}])

        return response.output_text
    
    def execute(self, decision: str):
        """Execute the chosen tool command."""

        try:
            if "(" in decision and ")" in decision:
                name, arg = decision.split("(",1)
                arg = arg.strip(")'\"")
                return self.tools.call(name.strip(),arg)
            else:
                return self.tools.call(decision)
        except Exception as e:
            return f"Error executing tool {e}"


        

### Register tools and run a demo

In [ ]:
registry = ToolRegistry()
registry.register("read_file", read_file)
registry.register("analyze_code",analyze_code)
registry.register("write_tests", write_tests)

assistant = CodeAssistantWithTools(registry)

user_request = "Please review the code in sample.py"

decision = assistant.decide_tool(user_input=user_request)
print(f"Agent Decision:\n{decision}")

result = assistant.execute(decision)
print(f"Tool Call Result:\n{result}")